# FINAL Submission Proyek Pengembangan Machine Learning Pipeline
Nama = Awang Mulya Nugrawan \
username Dicoding = awangnugrawan

# Import library

In [7]:
import pandas as pd
import os
from tfx.orchestration.beam.beam_dag_runner import BeamDagRunner
from modules import components, bank_market_pipelines

# 1. DATASET INFO

In [26]:
df = pd.read_csv('data\\bank.csv')
df

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
0,59,admin.,married,secondary,no,2343,yes,no,unknown,5,may,1042,1,-1,0,unknown,1
1,56,admin.,married,secondary,no,45,no,no,unknown,5,may,1467,1,-1,0,unknown,1
2,41,technician,married,secondary,no,1270,yes,no,unknown,5,may,1389,1,-1,0,unknown,1
3,55,services,married,secondary,no,2476,yes,no,unknown,5,may,579,1,-1,0,unknown,1
4,54,admin.,married,tertiary,no,184,no,no,unknown,5,may,673,2,-1,0,unknown,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11157,33,blue-collar,single,primary,no,1,yes,no,cellular,20,apr,257,1,-1,0,unknown,0
11158,39,services,married,secondary,no,733,no,no,unknown,16,jun,83,4,-1,0,unknown,0
11159,32,technician,single,secondary,no,29,no,no,cellular,19,aug,156,2,-1,0,unknown,0
11160,43,technician,married,secondary,no,0,no,yes,cellular,8,may,9,2,172,5,failure,0


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11162 entries, 0 to 11161
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        11162 non-null  int64 
 1   job        11162 non-null  object
 2   marital    11162 non-null  object
 3   education  11162 non-null  object
 4   default    11162 non-null  object
 5   balance    11162 non-null  int64 
 6   housing    11162 non-null  object
 7   loan       11162 non-null  object
 8   contact    11162 non-null  object
 9   day        11162 non-null  int64 
 10  month      11162 non-null  object
 11  duration   11162 non-null  int64 
 12  campaign   11162 non-null  int64 
 13  pdays      11162 non-null  int64 
 14  previous   11162 non-null  int64 
 15  poutcome   11162 non-null  object
 16  deposit    11162 non-null  int64 
dtypes: int64(8), object(9)
memory usage: 1.4+ MB


In [28]:
def count_unique_values(dataframe):
    categorical_columns = dataframe.select_dtypes(include=['object']).columns
    unique_counts = {}
    
    for column in categorical_columns:
        unique_values = dataframe[column].nunique()
        unique_counts[column] = unique_values
    
    return unique_counts

unique_counts = count_unique_values(df)
print(unique_counts)


{'job': 12, 'marital': 3, 'education': 4, 'default': 2, 'housing': 2, 'loan': 2, 'contact': 3, 'month': 12, 'poutcome': 4}


In [29]:
def print_numeric_columns(dataframe):
    numeric_columns = dataframe.select_dtypes(include=['int64']).columns
    
    for column in numeric_columns:
        print(column)

print_numeric_columns(df)


age
balance
day
duration
campaign
pdays
previous
deposit


In [30]:
df.isna().sum()

age          0
job          0
marital      0
education    0
default      0
balance      0
housing      0
loan         0
contact      0
day          0
month        0
duration     0
campaign     0
pdays        0
previous     0
poutcome     0
deposit      0
dtype: int64

# 2 Set Pipeline Variable

In [40]:
from tfx.orchestration.beam.beam_dag_runner import BeamDagRunner
from modules import components, bank_market_pipelines

In [41]:
PIPELANE_NAME = "awangnugrawan-pipeline"

# Pipeline inputs
DATA_ROOT = "data"
TRANSFORM_MODULE_FILE = "modules/bank_market_transform.py"
TUNER_MODULE_FILE = "modules/bank_market_tuner.py"
TRAINER_MODULE_FILE = "modules/bank_market_trainer.py"

# Pipeline outputs
OUTPUT_BASE = "outputs"

serving_model_dir = os.path.join(OUTPUT_BASE, "serving_model")
pipeline_root = os.path.join(OUTPUT_BASE, PIPELANE_NAME)
metadata_path = os.path.join(pipeline_root, "metadata.sqlite")

In [42]:
components_args = {
    "data_dir": DATA_ROOT,
    "training_module": TRAINER_MODULE_FILE,
    "tuner_module": TUNER_MODULE_FILE,
    "transform_module": TRANSFORM_MODULE_FILE,
    "training_steps": 500,
    "eval_steps": 200,
    "serving_model_dir": serving_model_dir,
}

components = components.init_components(components_args)

pipeline = bank_market_pipelines.init_pipeline(
    pipeline_root, 
    PIPELANE_NAME, 
    metadata_path, 
    components
)
BeamDagRunner().run(pipeline)

Trial 20 Complete [00h 00m 23s]
val_binary_accuracy: 0.7260156273841858

Best val_binary_accuracy So Far: 0.8510156273841858
Total elapsed time: 00h 06m 45s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


Results summary
Results in outputs\awangnugrawan-pipeline\Tuner\.system\executor_execution\7\.temp\7\bank_market_kt
Showing 10 best trials
Objective(name="val_binary_accuracy", direction="max")

Trial 07 summary
Hyperparameters:
num_layers: 1
dense_units: 128
dropout_rate: 0.1
learning_rate: 0.01
Score: 0.8510156273841858

Trial 14 summary
Hyperparameters:
num_layers: 2
dense_units: 192
dropout_rate: 0.4
learning_rate: 0.01
Score: 0.8503124713897705

Trial 13 summary
Hyperparameters:
num_layers: 1
dense_units: 208
dropout_rate: 0.30000000000000004
learning_rate: 0.01
Score: 0.8498437404632568

Trial 05 summary
Hyperparameters:
num_layers: 4
dense_units: 80
dropout_rate: 0.1
learning_rate: 0.01
Score: 0.84765625

Trial 08 summary
Hyperparameters:
num_layers: 1
dense_units: 224
dropout_rate: 0.1
learning_rate: 0.01
Score: 0.8463281393051147

Trial 15 summary
Hyperparameters:
num_layers: 1
dense_units: 192
dropout_rate: 0.4
learning_rate: 0.01
Score: 0.8461718559265137

Trial 18 summary
H

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 job_xf (InputLayer)            [(None, 13)]         0           []                               
                                                                                                  
 marital_xf (InputLayer)        [(None, 4)]          0           []                               
                                                                                                  
 education_xf (InputLayer)      [(None, 5)]          0           []                               
                                                                                                  
 default_xf (InputLayer)        [(None, 3)]          0           []                               
                                                                                            

INFO:tensorflow:Assets written to: outputs\awangnugrawan-pipeline\Trainer\model\8\Format-Serving\assets


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:Assets written to: outputs\awangnugrawan-pipeline\Trainer\model\8\Format-Serving\assets


INFO:tensorflow:Assets written to: outputs\awangnugrawan-pipeline\Trainer\model\8\Format-Serving\assets
